In [10]:
import pandas as pd
from bertopic import BERTopic

# Read the text file and store the sentences in a list
with open('SyntheticDataintext/1Allornothingthinking.txt', 'r') as file:
    sentences = file.readlines()


FileNotFoundError: [Errno 2] No such file or directory: 'SyntheticDataintext/1Allornothingthinking.txt'

In [9]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = list(set(stopwords.words('english')))
stop_words.extend([])
model = SentenceTransformer('all-MiniLM-L6-v2')
sentence_embeddings = model.encode(sentences,show_progress_bar=True)  # Has internal tokenzier
umap_model = UMAP(n_neighbors=25, n_components=8, min_dist=0, metric='cosine', random_state=42)

hdbscan_model = HDBSCAN(min_cluster_size=2, metric='cosine', cluster_selection_method='eom', prediction_data=True,cluster_selection_epsilon=0.18)

vectorizer_model = CountVectorizer(stop_words=stop_words,min_df=2, ngram_range=(1, 3))
topic_model = BERTopic(

    language="english",
    calculate_probabilities=True,

    # Pipeline models
    embedding_model=model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,

    # Hyperparameters
    top_n_words=5,
    verbose=True,
    )



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saura\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
c:\Users\saura\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


NameError: name 'sentences' is not defined

In [ ]:
topics, probs = topic_model.fit_transform(sentences, sentence_embeddings)

In [ ]:

# Initialize a BERTopic model
model = BERTopic()

# Fit the BERTopic model on the list of sentences
topics, _ = model.fit_transform(sentences)

# Find duplicate sentences within each topic
duplicate_sentences = {}
for topic_id in set(topics):
    topic_sentences = [sentences[i] for i, t in enumerate(topics) if t == topic_id]
    # To keep track of original sentences found in the duplicates
    seen = {}
    for i in range(len(topic_sentences)):
        for j in range(i + 1, len(topic_sentences)):
            if set(topic_sentences[i].lower().split()) == set(topic_sentences[j].lower().split()):
                if topic_sentences[i] not in seen and topic_sentences[j] not in seen:
                    seen[topic_sentences[i]] = topic_sentences[j]  # Set first as original
                elif topic_sentences[i] in seen:
                    seen[topic_sentences[j]] = topic_sentences[i]
                elif topic_sentences[j] in seen:
                    seen[topic_sentences[i]] = topic_sentences[j]

    duplicate_sentences[topic_id] = seen

# Create a new DataFrame to store the sentences and duplicate tags
df = pd.DataFrame({'Sentence': sentences, 'Duplicate': 'Original'})

# Iterate over the duplicate sentences, and tag them in the DataFrame
for topic_id, duplicates in duplicate_sentences.items():
    for dup, orig in duplicates.items():
        df.loc[df['Sentence'] == dup, 'Duplicate'] = f'Duplicate of {orig}'

# Print the DataFrame to check results
print(df)


In [24]:
# Save the DataFrame to a CSV file
df.to_csv('Allornothingthinkingduplicates.csv', index=False)

In [3]:
%pip install bertopic


^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia